In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats

import torch.optim as optim
import torch
import torch.nn as nn

from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
from ipywidgets import widgets
from IPython.display import display,clear_output

import warnings
warnings.filterwarnings("ignore")

In [3]:
from dataset import HelocDataset
from experiments import Benchmarking
from utils.logger_config import setup_logger
from tqdm import tqdm
from models.wrapper import PYTORCH_MODELS

logger = setup_logger()

In [4]:
from dataset import dataset_loader

In [5]:
name = 'heloc'
dataset_ares = dataset_loader(name, data_path='data/', dropped_features=[], n_bins=None)

In [6]:
from experiments.counterfactual import *
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.svm import SVC
from models import PyTorchDNN, PyTorchLinearSVM, PyTorchRBFNet, PyTorchLogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.gaussian_process import GaussianProcessClassifier

In [7]:
dataset = HelocDataset(dataset_ares=dataset_ares)

input_dim = dataset.get_dataframe().shape[1] - 1
seed = 0
np.random.seed(seed)
torch.manual_seed(seed)

counterfactual_algorithms = [
    'DiCE',
    'DisCount',
    # 'GlobeCE',
    # 'AReS',
    'KNN',
]

experiment = Benchmarking(
    dataset=dataset,
    models=[
        (BaggingClassifier(), 'sklearn'), 
        # (GaussianProcessClassifier(),'sklearn'),
        (PyTorchLogisticRegression(input_dim=input_dim), 'PYT'),
        (PyTorchDNN(input_dim=input_dim), 'PYT'),
        (PyTorchRBFNet(input_dim=input_dim, hidden_dim=input_dim), 'PYT'),
        # (PyTorchLinearSVM(input_dim=input_dim), 'PYT'),
        (RandomForestClassifier(), 'sklearn'), 
        (GradientBoostingClassifier(), 'sklearn'), 

        # (AdaBoostClassifier(), 'sklearn'), 
    ],
    shapley_methods=[
        "Train_Distri",
        "CF_UniformMatch",
        # "CF_ExactMatch",
        "CF_SingleMatch",
        "CF_OTMatch",
        "CF_OTMatch_2.0", 
        # "CF_OTMatch_50.0",
        # "CF_OTMatch_100.0",
        # "CF_OTMatch_10.0",
    ],
    distance_metrics=[
        'optimal_transport',
        'mean_difference',
        'median_difference',
        'max_mean_discrepancy',
    ],
    md_baseline = False
)

experiment.train_and_evaluate_models(random_state=seed)
experiment.models_performance()

logger.info("\n\n------Compute Counterfactuals------")
sample_num = 100
model_counterfactuals = {}
for model, model_name in zip(experiment.models, experiment.model_names):
    model_counterfactuals[model_name] = {}

    for algorithm in counterfactual_algorithms:
        if algorithm == 'DisCount' and model_name not in PYTORCH_MODELS:
            logger.info(f'Skipping {algorithm} for {model_name} due to incompatability')
            continue
        logger.info(f'Computing {model_name} counterfactuals with {algorithm}')
        function_name = f"compute_{algorithm}_counterfactuals"
        try:
            func = globals()[function_name]
            model_counterfactuals[model_name][algorithm] = func(
                experiment.X_test,
                model = model,
                target_name = experiment.dataset.target_name,
                sample_num = sample_num,
                experiment=experiment,
            )
        except KeyError:
            print(f"Function {function_name} is not defined.")

INFO:root:BaggingClassifier accuracy: 0.6967088607594937
INFO:root:PyTorchLogisticRegression accuracy: 0.6967088607594937
INFO:root:PyTorchDNN accuracy: 0.6967088607594937
INFO:root:PyTorchRBFNet accuracy: 0.6967088607594937
INFO:root:RandomForestClassifier accuracy: 0.6967088607594937
INFO:root:GradientBoostingClassifier accuracy: 0.6967088607594937
INFO:root:

------Compute Counterfactuals------
INFO:root:Computing BaggingClassifier counterfactuals with DiCE
100%|██████████| 100/100 [03:47<00:00,  2.27s/it]
INFO:root:Skipping DisCount for BaggingClassifier due to incompatability
INFO:root:Computing BaggingClassifier counterfactuals with KNN
INFO:root:Computing PyTorchLogisticRegression counterfactuals with DiCE
100%|██████████| 100/100 [01:24<00:00,  1.19it/s]
INFO:root:Computing PyTorchLogisticRegression counterfactuals with DisCount
INFO:root:Optimization started
100%|██████████| 50/50 [20:23<00:00, 24.47s/it]
INFO:root:Computing PyTorchLogisticRegression counterfactuals with KNN
I

In [ ]:
warnings.filterwarnings("ignore", category=DeprecationWarning)

logger.info("\n\n------Compute Shapley Values------")
experiment.compute_intervention_policies(
    model_counterfactuals=model_counterfactuals,
);

In [ ]:
experiment.distance_metrics = [
    'optimal_transport',
    'mean_difference',
    'median_difference',
    'max_mean_discrepancy',
]

In [ ]:
logger.info("\n\n------Evaluating Distance Performance Under Interventions------")
experiment.evaluate_distance_performance_under_interventions(
    intervention_num_list=range(0,801,20),
    trials_num=100,
    replace=False,
)

In [ ]:
from experiments import plotting

plotting.intervention_vs_distance(experiment, save_to_file=False)

In [ ]:
import pickle
with open(f"pickles/{dataset.name}_distance_results.pickle", "wb") as output_file:
    pickle.dump(experiment.distance_results, output_file)

In [ ]:
data = experiment.distance_results['PyTorchDNN']['KNN']

In [ ]:
data['CF_UniformMatch'].keys()

In [ ]:
from experiments.latex import TikzPlotGenerator

In [ ]:
plot_generator = TikzPlotGenerator(data)
print(plot_generator.generate_plot_code(
    metric='median_difference', 
    methods=[
        "Train_Distri",
        'CF_UniformMatch',
        'CF_SingleMatch',
        'CF_OTMatch_2.0',
    ]
))